In [ ]:
from selenium import webdriver
import pandas as pd
import re


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException
import time

chrome_driver_path = r"D:\chromedriver-win64\chromedriver-win64\chromedriver.exe"
driver = webdriver.Chrome(service=Service(chrome_driver_path))
url = "https://aiqicha.baidu.com/"
driver.get(url)

In [1]:
#老版本，有问题
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time

def check_hospital_public_status(hospital_name):
    options = Options()
    options.headless = True  # 无头模式
    driver = webdriver.Chrome(options=options)

    try:
        driver.get("https://aiqicha.baidu.com/")
        search_box = driver.find_element(By.ID, 'aqc-search-input')
        search_box.send_keys(hospital_name)
        search_box.send_keys(Keys.RETURN)
        time.sleep(5)  # 给予足够时间以加载结果

        # 查找“事业单位”字样
        result_element = driver.find_element(By.XPATH, "//*[contains(text(), '事业单位')]")
        for result in results:
            if hospital_name in result.text:
                # 在匹配的结果区域内查找“事业单位”
                if "事业单位" in result.text:
                    return "公立1"
                else:
                    return "私立"
        return "找不到匹配项"  # 如果没有找到匹配的医院名称
    except Exception as e:
        print(f"Error processing {hospital_name}: {e}")
        return "找不到匹配项"
    finally:
        driver.quit()


def process_hospitals(file_path1,file_path2):
    # 读取Excel文件
    df = pd.read_excel(file_path1)
    # 假设医院名字在名为'Hospital Name'的列
    df['Public Status'] = df['Hospital Name'].apply(check_hospital_public_status)
    # 将结果保存回Excel
    df.to_excel(file_path2, index=False)

# 用文件路径替换这里的'path_to_your_excel.xlsx'
file_path1 = r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-24\test_input.xlsx'
file_path2 = r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-24\test_output.xlsx'
process_hospitals(file_path1,file_path2)


Error processing 大同区林源医院: name 'results' is not defined
Error processing 齐齐哈尔爱民医院: name 'results' is not defined
Error processing 大庆眼科医院: name 'results' is not defined
Error processing 大庆妇科医院: name 'results' is not defined
Error processing 大庆市萨尔图区东风新村中医院: name 'results' is not defined
Error processing 呼玛县人民医院: name 'results' is not defined


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\cxy\\OneDrive\\桌面\\回国\\实习\\7-24\\test_output.xlsx'

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd

def setup_driver():
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get("https://aiqicha.baidu.com/")
    return driver

def check_hospital_public_status(driver, hospital_name):
    try:
        search_box = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'aqc-search-input'))
        )
        search_box.clear()
        search_box.send_keys(hospital_name)
        search_box.send_keys(Keys.RETURN)
        
        # 等待搜索结果
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".search-result-single"))
        )
        results = driver.find_elements(By.CSS_SELECTOR, ".search-result-single")
        for result in results:
            if hospital_name in result.text:
                if "事业单位" in result.text:
                    return "公立"
                return "私立"
        return "找不到匹配项"
    except TimeoutException:
        return "搜索超时"
    except NoSuchElementException:
        return "元素未找到"
    except Exception as e:
        print(f"Error processing {hospital_name}: {e}")
        return "查询出错"

def process_hospitals(file_path1, file_path2):
    driver = setup_driver()
    df = pd.read_excel(file_path1)
    df['Public Status'] = df['Hospital Name'].apply(lambda x: check_hospital_public_status(driver, x))
    df.to_excel(file_path2, index=False)
    driver.quit()

# 文件路径
file_path1 = r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-24\test_input.xlsx'
file_path2 = r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-24\test_output.xlsx'
process_hospitals(file_path1, file_path2)



In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import difflib

def check_hospital_public_status(hospital_name):
    options = Options()
    options.headless = True  # Enable headless mode
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 3)

    try:
        driver.get("https://aiqicha.baidu.com/")
        search_box = wait.until(EC.element_to_be_clickable((By.ID, 'aqc-search-input')))
        search_box.send_keys(hospital_name + Keys.RETURN)
        
        # Wait for the results to load
        results = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.header.company-list-wrap.card .info-items .title')))
        
        # Check if the first result's name similarity is above 90%
        result_name = results[0].text
        similarity = difflib.SequenceMatcher(None, hospital_name, result_name).ratio()
        
        if similarity > 0.9:
            tag_list = driver.find_elements(By.CSS_SELECTOR, '.header.company-list-wrap.card .tags-list')[0]
            tags = tag_list.find_elements(By.TAG_NAME, 'span')
            for tag in tags:
                if '事业单位' in tag.text:
                    return "公立"
            return "私立"
        else:
            return "未找到"
    except Exception as e:
        print(f"Error processing {hospital_name}: {e}")
        return "未找到"
    finally:
        driver.quit()

def process_hospitals(file_path1, file_path2):
    df = pd.read_excel(file_path1)
    df['Public Status'] = df['Hospital Name'].apply(check_hospital_public_status)
    df.to_excel(file_path2, index=False)

file_path1 = r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-24\test_input.xlsx'
file_path2 = r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-24\test_output.xlsx'
process_hospitals(file_path1, file_path2)



Error processing 大同区林源医院: Message: 

Error processing 齐齐哈尔爱民医院: Message: 

Error processing 大庆眼科医院: Message: 

Error processing 大庆妇科医院: Message: 

Error processing 大庆市萨尔图区东风新村中医院: Message: 

Error processing 呼玛县人民医院: Message: 



In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def check_hospital_public_status(hospital_name):
    options = Options()
    options.headless = True  # 无头模式
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)

    try:
        driver.get("https://aiqicha.baidu.com/")
        search_box = wait.until(EC.element_to_be_clickable((By.ID, 'aqc-search-input')))
        search_box.send_keys(hospital_name + Keys.RETURN)
        
        # 使用显式等待代替固定的sleep时间
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.header.company-list-wrap.card')))
        
        # 尝试查找含有“事业单位”的元素
        try:
            result_element = wait.until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), '事业单位')]")))
            return "公立" if "事业单位" in result_element.text else "私立"
        except:
            return "私立"  # 如果找不到“事业单位”则假定为私立
        
    except Exception as e:
        print(f"Error processing {hospital_name}: {e}")
        return "找不到匹配项"
    finally:
        driver.quit()

def process_hospitals(file_path1, file_path2):
    df = pd.read_excel(file_path1)
    df['Public Status'] = df['Hospital Name'].apply(check_hospital_public_status)
    df.to_excel(file_path2, index=False)

file_path1 = r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-24\test_input.xlsx'
file_path2 = r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-24\test_output.xlsx'
process_hospitals(file_path1, file_path2)


Error processing 大同区林源医院: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF645E99632+30946]
	(No symbol) [0x00007FF645E4E3C9]
	(No symbol) [0x00007FF645D46FDA]
	(No symbol) [0x00007FF645D9822C]
	(No symbol) [0x00007FF645D9850C]
	(No symbol) [0x00007FF645DDDCB7]
	(No symbol) [0x00007FF645DBCAAF]
	(No symbol) [0x00007FF645DDB041]
	(No symbol) [0x00007FF645DBC813]
	(No symbol) [0x00007FF645D8A6E5]
	(No symbol) [0x00007FF645D8B021]
	GetHandleVerifier [0x00007FF645FCF83D+1301229]
	GetHandleVerifier [0x00007FF645FDBDB7+1351783]
	GetHandleVerifier [0x00007FF645FD2A03+1313971]
	GetHandleVerifier [0x00007FF645ECDD06+245686]
	(No symbol) [0x00007FF645E5758F]
	(No symbol) [0x00007FF645E53804]
	(No symbol) [0x00007FF645E53992]
	(No symbol) [0x00007FF645E4A3EF]
	BaseThreadInitThunk [0x00007FFF503D257D+29]
	RtlUserThreadStart [0x00007FFF51A0AF28+40]

Error processing 齐齐哈尔爱民医院: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0